In [ ]:
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.ensemble import RandomForestClassifier
import sklearn
from sklearn.metrics import classification_report

import wrangle
import explore
from model import model_df
pd.set_option('display.max_columns', None)

## Acquiring the data

In [ ]:
df = pd.read_csv('service_calls.csv')

## Preparing the data

In [ ]:
df = wrangle.clean_311(df)

In [ ]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled = wrangle.split_separate_scale(df)

In [ ]:
# Previewing the data

df.head()

In [ ]:
df.describe()